In [1]:
# !pip3 install -U scikit-learn scipy matplotlib
# !pip3 install matplotlib
# !pip3 install pandas

In [2]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np
import multiprocessing as mc
import time
import random
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import random as rnd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [3]:
mc.cpu_count()

4

In [4]:
data = pd.read_csv('dataset.csv')

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\3991704923.py:1: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('dataset.csv')


In [5]:
data.dtypes

transact                       int64
onServiceConnected             int64
bindService                    int64
attachInterface                int64
ServiceConnection              int64
                               ...  
ACCESS_FINE_LOCATION           int64
SET_WALLPAPER_HINTS            int64
SET_PREFERRED_APPLICATIONS     int64
WRITE_SECURE_SETTINGS          int64
class                         object
Length: 216, dtype: object

In [6]:
classes,count = np.unique(data['class'],return_counts=True)
#Perform Label Encoding
lbl_enc = LabelEncoder()
print(lbl_enc.fit_transform(classes),classes)
data = data.replace(classes,lbl_enc.fit_transform(classes))

#Dataset contains special characters like ''?' and 'S'. Set them to NaN and use dropna() to remove them
data=data.replace('[?,S]',np.NaN,regex=True)
print("Total missing values : ",sum(list(data.isna().sum())))
data.dropna(inplace=True)
for c in data.columns:
    data[c] = pd.to_numeric(data[c])
data

[0 1] ['B' 'S']
Total missing values :  5


,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,...,READ_CONTACTS,DEVICE_POWER,HARDWARE_TEST,ACCESS_WIFI_STATE,WRITE_EXTERNAL_STORAGE,ACCESS_FINE_LOCATION,SET_WALLPAPER_HINTS,SET_PREFERRED_APPLICATIONS,WRITE_SECURE_SETTINGS,class
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,1,1,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15031,1,1,1,1,1,1,0,1,1,1,...,0,0,0,1,1,0,0,0,0,0
15032,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,1,0,0,0,0,0
15033,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
15034,1,1,1,1,1,1,0,1,1,1,...,1,0,0,1,1,1,0,0,0,0


In [7]:
y = data['class'].copy()
data = data.drop(columns = 'class')

In [8]:
data1 = data.iloc[:, :19]
data1['class'] = pd.DataFrame(y)
data1.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\3723814616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['class'] = pd.DataFrame(y)


,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,Ljava.net.URLDecoder,android.content.pm.Signature,android.telephony.SmsManager,READ_PHONE_STATE,getBinder,ClassLoader,Landroid.content.Context.registerReceiver,Ljava.lang.Class.getField,Landroid.content.Context.unregisterReceiver,class
transact,1.000000,0.810763,0.816395,0.963355,0.813133,0.881907,-0.274939,0.656701,0.587034,0.613710,0.570387,0.643977,-0.206015,-0.081029,0.492964,0.577761,0.555345,0.569698,0.547951,-0.570093
onServiceConnected,0.810763,1.000000,0.983194,0.783564,0.984792,0.842648,-0.324744,0.644692,0.596599,0.593575,0.570201,0.626964,-0.244261,-0.058871,0.504727,0.569465,0.559273,0.554240,0.556690,-0.561036
bindService,0.816395,0.983194,1.000000,0.791157,0.994620,0.848676,-0.321889,0.649393,0.599701,0.595670,0.581903,0.633114,-0.240018,-0.059110,0.511034,0.568819,0.558883,0.550730,0.556410,-0.558808
attachInterface,0.963355,0.783564,0.791157,1.000000,0.787746,0.861405,-0.264026,0.642582,0.580355,0.608874,0.548505,0.636882,-0.195830,-0.069642,0.502550,0.556924,0.541610,0.548603,0.544839,-0.551434
ServiceConnection,0.813133,0.984792,0.994620,0.787746,1.000000,0.848090,-0.322683,0.646712,0.599214,0.594027,0.579999,0.629682,-0.241031,-0.057555,0.508577,0.568606,0.556957,0.550155,0.552741,-0.557568
android.os.Binder,0.881907,0.842648,0.848676,0.861405,0.848090,1.000000,-0.304983,0.611953,0.539541,0.576761,0.530415,0.580833,-0.230374,-0.075223,0.448844,0.535882,0.537601,0.525050,0.534753,-0.553608
SEND_SMS,-0.274939,-0.324744,-0.321889,-0.264026,-0.322683,-0.304983,1.000000,-0.203369,-0.215223,-0.245969,-0.261750,-0.185176,0.754027,0.309440,-0.159667,-0.269432,-0.139824,-0.156140,-0.123209,0.546075
Ljava.lang.Class.getCanonicalName,0.656701,0.644692,0.649393,0.642582,0.646712,0.611953,-0.203369,1.000000,0.712875,0.729435,0.638553,0.708556,-0.130134,-0.064135,0.581476,0.586303,0.570458,0.694142,0.547137,-0.467567
Ljava.lang.Class.getMethods,0.587034,0.596599,0.599701,0.580355,0.599214,0.539541,-0.215223,0.712875,1.000000,0.686083,0.607434,0.688524,-0.165342,-0.066557,0.551573,0.573189,0.504482,0.669578,0.501471,-0.436322
Ljava.lang.Class.cast,0.613710,0.593575,0.595670,0.608874,0.594027,0.576761,-0.245969,0.729435,0.686083,1.000000,0.622730,0.661834,-0.169478,-0.065294,0.582011,0.558529,0.561500,0.613153,0.514096,-0.440908


In [9]:
data2 = data.iloc[:, 19:38]
data2['class'] = pd.DataFrame(y)
data2.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\3721455631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['class'] = pd.DataFrame(y)


,GET_ACCOUNTS,RECEIVE_SMS,Ljava.lang.Class.getDeclaredField,READ_SMS,getCallingUid,Ljavax.crypto.spec.SecretKeySpec,android.intent.action.BOOT_COMPLETED,USE_CREDENTIALS,MANAGE_ACCOUNTS,android.content.pm.PackageInfo,KeySpec,TelephonyManager.getLine1Number,DexClassLoader,HttpGet.init,SecretKey,Ljava.lang.Class.getMethod,System.loadLibrary,android.intent.action.SEND,Ljavax.crypto.Cipher,class
GET_ACCOUNTS,1.000000,-0.092386,0.446846,-0.038097,0.286040,0.340202,0.025675,0.487726,0.516942,0.338867,0.329467,0.038304,0.290177,0.360249,0.325700,0.374664,0.242083,0.317966,0.316512,-0.366473
RECEIVE_SMS,-0.092386,1.000000,-0.181552,0.570736,-0.089425,-0.191610,0.284841,-0.029169,0.006149,-0.172750,-0.172754,0.175246,-0.023124,-0.120885,-0.168186,-0.050149,-0.105919,0.006731,-0.151706,0.388328
Ljava.lang.Class.getDeclaredField,0.446846,-0.181552,1.000000,-0.126727,0.308549,0.496905,-0.010490,0.244637,0.234313,0.471108,0.477933,-0.038062,0.436113,0.516154,0.467165,0.599990,0.350431,0.248801,0.444888,-0.367717
READ_SMS,-0.038097,0.570736,-0.126727,1.000000,-0.053091,-0.119538,0.325806,-0.043570,0.007621,-0.071220,-0.111754,0.098884,0.005340,-0.184208,-0.107105,-0.065583,-0.013083,0.014693,-0.097229,0.370336
getCallingUid,0.286040,-0.089425,0.308549,-0.053091,1.000000,0.297329,-0.014204,0.261250,0.276236,0.240708,0.282099,-0.019922,0.411280,0.277737,0.278031,0.264067,0.184695,0.182493,0.284462,-0.281609
Ljavax.crypto.spec.SecretKeySpec,0.340202,-0.191610,0.496905,-0.119538,0.297329,1.000000,0.043902,0.161282,0.164744,0.509128,0.919946,0.106209,0.418000,0.526921,0.943356,0.444925,0.393692,0.163964,0.880300,-0.325141
android.intent.action.BOOT_COMPLETED,0.025675,0.284841,-0.010490,0.325806,-0.014204,0.043902,1.000000,0.015131,0.045058,0.010694,0.052394,0.225976,-0.013469,-0.000916,0.062266,0.019417,0.015927,0.014651,0.066010,0.314303
USE_CREDENTIALS,0.487726,-0.029169,0.244637,-0.043570,0.261250,0.161282,0.015131,1.000000,0.723771,0.171408,0.154785,0.008848,0.115113,0.214280,0.155936,0.213399,0.136130,0.294259,0.158757,-0.255436
MANAGE_ACCOUNTS,0.516942,0.006149,0.234313,0.007621,0.276236,0.164744,0.045058,0.723771,1.000000,0.196471,0.167074,0.044985,0.102187,0.211108,0.161487,0.228289,0.175786,0.384618,0.147227,-0.256503
android.content.pm.PackageInfo,0.338867,-0.172750,0.471108,-0.071220,0.240708,0.509128,0.010694,0.171408,0.196471,1.000000,0.515216,0.081028,0.339786,0.474356,0.501036,0.482795,0.401511,0.210150,0.480432,-0.310313


In [10]:
data3 = data.iloc[:, 38:57]
data3['class'] = pd.DataFrame(y)
data3.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\2966802453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['class'] = pd.DataFrame(y)


,WRITE_SMS,READ_SYNC_SETTINGS,AUTHENTICATE_ACCOUNTS,android.telephony.gsm.SmsManager,WRITE_HISTORY_BOOKMARKS,TelephonyManager.getSubscriberId,mount,INSTALL_PACKAGES,Runtime.getRuntime,CAMERA,Ljava.lang.Object.getClass,WRITE_SYNC_SETTINGS,READ_HISTORY_BOOKMARKS,Ljava.lang.Class.forName,INTERNET,android.intent.action.PACKAGE_REPLACED,Binder,android.intent.action.SEND_MULTIPLE,RECORD_AUDIO,class
WRITE_SMS,1.000000,0.032088,-0.053551,0.247507,0.048865,0.297530,-0.003069,0.089600,0.042684,0.061993,0.079690,0.002175,0.096038,-0.066785,0.099007,0.015649,0.098881,0.025982,0.037615,0.267501
READ_SYNC_SETTINGS,0.032088,1.000000,0.612411,-0.042675,0.006796,-0.018242,0.138340,-0.032676,0.148177,0.248778,0.152857,0.885807,0.014625,0.159745,0.097320,0.110978,0.122291,0.372742,0.271332,-0.215829
AUTHENTICATE_ACCOUNTS,-0.053551,0.612411,1.000000,-0.042003,-0.067007,-0.002030,0.127948,-0.022043,0.173724,0.197052,0.139038,0.632327,-0.066246,0.156518,0.083348,0.113865,0.128792,0.285473,0.221880,-0.200315
android.telephony.gsm.SmsManager,0.247507,-0.042675,-0.042003,1.000000,0.051597,0.118714,-0.055396,0.175367,-0.052996,-0.027306,-0.097990,0.003207,0.041312,-0.047542,0.053341,-0.028070,0.002453,-0.025988,-0.027218,0.241038
WRITE_HISTORY_BOOKMARKS,0.048865,0.006796,-0.067007,0.051597,1.000000,0.063243,0.145061,0.008203,-0.095062,0.007864,0.119726,0.036915,0.912184,0.148395,0.099089,-0.037684,-0.066519,-0.055924,-0.039251,0.242250
TelephonyManager.getSubscriberId,0.297530,-0.018242,-0.002030,0.118714,0.063243,1.000000,0.081262,0.178292,0.124901,-0.012504,0.164415,0.009633,0.066771,0.062641,0.181807,0.061038,0.183576,-0.041860,-0.015651,0.241551
mount,-0.003069,0.138340,0.127948,-0.055396,0.145061,0.081262,1.000000,-0.109390,0.541801,0.211107,0.547624,0.123252,0.141631,0.559119,0.295670,0.104489,0.350781,0.150991,0.175167,-0.233526
INSTALL_PACKAGES,0.089600,-0.032676,-0.022043,0.175367,0.008203,0.178292,-0.109390,1.000000,-0.067672,-0.034738,0.002411,-0.043126,0.030739,0.005952,0.086230,0.043863,0.082562,-0.053387,-0.054572,0.235660
Runtime.getRuntime,0.042684,0.148177,0.173724,-0.052996,-0.095062,0.124901,0.541801,-0.067672,1.000000,0.248381,0.455672,0.139833,-0.081829,0.464423,0.275035,0.163625,0.367376,0.188863,0.167625,-0.228695
CAMERA,0.061993,0.248778,0.197052,-0.027306,0.007864,-0.012504,0.211107,-0.034738,0.248381,1.000000,0.150013,0.226991,0.048097,0.182499,0.101182,0.141524,0.129111,0.284673,0.376841,-0.211914


In [11]:
data4 = data.iloc[:, 57:76]
data4['class'] = pd.DataFrame(y)
data4.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\3782231556.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['class'] = pd.DataFrame(y)


,IBinder,android.os.IBinder,createSubprocess,NFC,ACCESS_LOCATION_EXTRA_COMMANDS,URLClassLoader,WRITE_APN_SETTINGS,abortBroadcast,BIND_REMOTEVIEWS,android.intent.action.TIME_SET,READ_PROFILE,TelephonyManager.getDeviceId,MODIFY_AUDIO_SETTINGS,getCallingPid,READ_SYNC_STATS,BROADCAST_STICKY,android.intent.action.PACKAGE_REMOVED,android.intent.action.TIMEZONE_CHANGED,WAKE_LOCK,class
IBinder,1.000000,0.999803,0.061469,0.089981,0.008079,0.112257,0.091280,0.120636,0.095495,0.088849,0.034595,0.295816,0.053777,0.100581,0.094865,0.088761,0.106552,0.062188,0.225730,-0.200537
android.os.IBinder,0.999803,1.000000,0.060349,0.090005,0.007386,0.112281,0.090546,0.120168,0.095516,0.088874,0.033741,0.295958,0.053813,0.099772,0.094885,0.088788,0.105923,0.062209,0.225539,-0.200390
createSubprocess,0.061469,0.060349,1.000000,-0.030124,-0.025549,-0.032579,0.404792,0.250085,-0.028594,-0.029287,-0.025107,0.068732,-0.032660,-0.024686,-0.027516,-0.033752,-0.007635,-0.022889,0.074262,0.185971
NFC,0.089981,0.090005,-0.030124,1.000000,-0.036370,0.027282,-0.013227,0.016325,0.187280,0.018031,0.102813,-0.061644,0.116367,0.092005,0.068858,0.108395,0.102589,-0.013033,0.122781,-0.157983
ACCESS_LOCATION_EXTRA_COMMANDS,0.008079,0.007386,-0.025549,-0.036370,1.000000,-0.031308,-0.001416,-0.047109,-0.041237,-0.020324,-0.018323,0.186618,0.024718,-0.032018,-0.024551,0.017906,-0.047848,-0.032436,0.082452,0.197165
URLClassLoader,0.112257,0.112281,-0.032579,0.027282,-0.031308,1.000000,-0.041261,-0.012671,0.090880,0.005511,0.089798,0.074611,0.064735,0.116144,0.088916,0.052655,0.000945,-0.004867,0.100139,-0.161923
WRITE_APN_SETTINGS,0.091280,0.090546,0.404792,-0.013227,-0.001416,-0.041261,1.000000,0.424474,-0.038976,0.022470,-0.028856,0.126121,0.013370,-0.014795,-0.009757,-0.005010,-0.003741,0.008199,0.120769,0.193827
abortBroadcast,0.120636,0.120168,0.250085,0.016325,-0.047109,-0.012671,0.424474,1.000000,0.050081,0.008124,0.069940,0.169729,0.122138,0.036985,0.060540,0.058822,0.055174,-0.001149,0.149704,0.191727
BIND_REMOTEVIEWS,0.095495,0.095516,-0.028594,0.187280,-0.041237,0.090880,-0.038976,0.050081,1.000000,0.084436,0.182610,0.026806,0.099077,0.097131,0.129991,0.010124,0.064392,0.072167,0.142006,-0.150617
android.intent.action.TIME_SET,0.088849,0.088874,-0.029287,0.018031,-0.020324,0.005511,0.022470,0.008124,0.084436,1.000000,0.033901,0.024787,0.040368,0.219616,0.083636,0.098224,0.160294,0.709052,0.128782,-0.156175


In [12]:
data5 = data.iloc[:, 76:95]
data5['class'] = pd.DataFrame(y)
data5.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\3575328896.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data5['class'] = pd.DataFrame(y)


,RECEIVE_BOOT_COMPLETED,RESTART_PACKAGES,Ljava.lang.Class.getPackage,chmod,Ljava.lang.Class.getDeclaredClasses,android.intent.action.ACTION_POWER_DISCONNECTED,android.intent.action.PACKAGE_ADDED,PathClassLoader,TelephonyManager.getSimSerialNumber,Runtime.load,TelephonyManager.getCallState,BLUETOOTH,READ_CALENDAR,READ_CALL_LOG,SUBSCRIBED_FEEDS_WRITE,READ_EXTERNAL_STORAGE,TelephonyManager.getSimCountryIso,sendMultipartTextMessage,PackageInstaller,class
RECEIVE_BOOT_COMPLETED,1.000000,0.190307,0.128254,0.125757,0.012890,0.092182,0.125495,0.036447,0.092430,0.043953,0.156035,0.146883,0.100467,0.120329,0.016888,0.080926,0.068785,0.104196,0.106007,0.159870
RESTART_PACKAGES,0.190307,1.000000,-0.014964,0.032374,0.001343,0.076275,0.095426,-0.023055,0.032933,-0.013333,0.104525,0.204242,0.009709,0.043791,-0.022165,-0.030838,0.097729,0.058169,0.103361,0.157646
Ljava.lang.Class.getPackage,0.128254,-0.014964,1.000000,0.009124,0.139120,0.086719,0.038964,0.130796,-0.033488,0.076124,0.068639,0.107106,0.043757,0.033112,0.027285,0.111501,0.202687,0.019515,-0.042665,-0.148418
chmod,0.125757,0.032374,0.009124,1.000000,0.024607,0.002790,0.092745,0.058861,0.176551,0.045044,0.064688,0.017417,-0.027988,0.062346,-0.034044,0.114911,0.018748,0.057843,0.080150,0.146465
Ljava.lang.Class.getDeclaredClasses,0.012890,0.001343,0.139120,0.024607,1.000000,0.055860,0.056998,0.123865,-0.014973,0.075683,0.128636,0.028632,0.015909,0.026550,0.026693,0.057058,0.101407,-0.001390,-0.015109,-0.120603
android.intent.action.ACTION_POWER_DISCONNECTED,0.092182,0.076275,0.086719,0.002790,0.055860,1.000000,0.206671,0.021317,0.055392,-0.000424,0.105876,0.136782,0.047196,0.161497,0.145815,0.094054,0.100321,0.108666,0.044714,-0.120603
android.intent.action.PACKAGE_ADDED,0.125495,0.095426,0.038964,0.092745,0.056998,0.206671,1.000000,0.054331,0.099261,-0.009258,0.165956,0.159666,0.028075,0.185538,0.160001,0.061886,0.146232,0.210901,0.102243,-0.132396
PathClassLoader,0.036447,-0.023055,0.130796,0.058861,0.123865,0.021317,0.054331,1.000000,-0.042351,0.552380,0.145911,0.003549,0.096525,-0.020882,0.022461,0.214801,0.100078,0.003316,-0.016050,-0.116635
TelephonyManager.getSimSerialNumber,0.092430,0.032933,-0.033488,0.176551,-0.014973,0.055392,0.099261,-0.042351,1.000000,-0.024244,0.115881,-0.000251,-0.031881,0.087518,0.018546,0.020063,0.112603,0.079766,0.181201,0.135075
Runtime.load,0.043953,-0.013333,0.076124,0.045044,0.075683,-0.000424,-0.009258,0.552380,-0.024244,1.000000,0.208078,0.017009,0.153647,0.050141,-0.018137,0.185501,0.099436,0.056699,-0.024742,-0.105350


In [13]:
data6 = data.iloc[:, 95:114]
data6['class'] = pd.DataFrame(y)
data6.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\2612950097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data6['class'] = pd.DataFrame(y)


,VIBRATE,remount,android.intent.action.ACTION_SHUTDOWN,sendDataMessage,ACCESS_NETWORK_STATE,chown,HttpPost.init,Ljava.lang.Class.getClasses,SUBSCRIBED_FEEDS_READ,TelephonyManager.isNetworkRoaming,CHANGE_WIFI_MULTICAST_STATE,WRITE_CALENDAR,android.intent.action.PACKAGE_DATA_CLEARED,MASTER_CLEAR,HttpUriRequest,UPDATE_DEVICE_STATS,WRITE_CALL_LOG,DELETE_PACKAGES,GET_TASKS,class
VIBRATE,1.000000,0.000767,0.070804,-0.011369,0.290075,-0.014475,0.223116,0.074929,0.066820,0.174649,0.030874,0.104492,0.049828,0.082303,0.168377,-0.049329,0.056482,-0.056810,0.202598,-0.109582
remount,0.000767,1.000000,0.014034,0.089712,0.047705,0.686306,-0.010287,0.040365,-0.020760,0.107784,0.035450,-0.005628,0.012323,-0.017682,-0.028371,-0.019575,0.079406,-0.019242,0.036179,0.112943
android.intent.action.ACTION_SHUTDOWN,0.070804,0.014034,1.000000,-0.002982,0.040031,-0.014873,0.029263,0.045050,-0.018288,0.072679,0.005655,0.034948,0.190965,0.063791,0.019964,-0.009257,0.201161,0.040662,0.134792,-0.094419
sendDataMessage,-0.011369,0.089712,-0.002982,1.000000,-0.035838,-0.015026,0.044199,0.021476,0.079029,0.095355,-0.016033,0.008635,-0.014777,0.063162,0.051164,0.014377,0.019898,-0.000685,0.017944,0.110062
ACCESS_NETWORK_STATE,0.290075,0.047705,0.040031,-0.035838,1.000000,0.044125,0.451230,0.088940,0.005931,0.138450,0.068071,0.038116,0.040758,-0.029293,0.430349,0.059049,0.048936,0.068313,0.215557,-0.106194
chown,-0.014475,0.686306,-0.014873,-0.015026,0.044125,1.000000,0.007247,0.038057,-0.016980,0.115370,-0.010149,-0.005773,-0.013626,-0.014463,-0.000221,-0.016011,0.029012,-0.018961,0.016357,0.107540
HttpPost.init,0.223116,-0.010287,0.029263,0.044199,0.451230,0.007247,1.000000,0.086997,0.079795,0.087850,0.035143,0.045472,0.030106,-0.072094,0.815743,-0.077325,0.046517,-0.066048,0.207545,-0.105325
Ljava.lang.Class.getClasses,0.074929,0.040365,0.045050,0.021476,0.088940,0.038057,0.086997,1.000000,-0.023201,0.094028,-0.004467,0.033864,0.045786,-0.012300,0.086792,-0.015979,0.082530,0.119295,0.078396,-0.097075
SUBSCRIBED_FEEDS_READ,0.066820,-0.020760,-0.018288,0.079029,0.005931,-0.016980,0.079795,-0.023201,1.000000,0.075757,-0.009838,0.085324,0.129374,0.106043,0.070096,-0.007038,-0.002130,-0.018556,-0.022665,-0.088124
TelephonyManager.isNetworkRoaming,0.174649,0.107784,0.072679,0.095355,0.138450,0.115370,0.087850,0.094028,0.075757,1.000000,0.041024,0.048777,0.130417,0.081618,0.089296,0.008866,0.117306,0.033793,0.107973,-0.096821


In [14]:
data7 = data.iloc[:, 114:133]
data7['class'] = pd.DataFrame(y)
data7.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\815819063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data7['class'] = pd.DataFrame(y)


,GLOBAL_SEARCH,DELETE_CACHE_FILES,WRITE_USER_DICTIONARY,android.intent.action.PACKAGE_CHANGED,android.intent.action.NEW_OUTGOING_CALL,REORDER_TASKS,WRITE_PROFILE,SET_WALLPAPER,BIND_INPUT_METHOD,divideMessage,READ_SOCIAL_STREAM,READ_USER_DICTIONARY,PROCESS_OUTGOING_CALLS,CALL_PRIVILEGED,Runtime.exec,BIND_WALLPAPER,RECEIVE_WAP_PUSH,DUMP,BATTERY_STATS,class
GLOBAL_SEARCH,1.000000,-0.012915,0.043064,0.183920,-0.017049,0.009541,0.142314,0.132153,0.057332,0.083284,0.153421,0.048698,-0.021983,0.017846,0.110172,-0.030765,0.167931,0.139039,-0.023113,-0.082356
DELETE_CACHE_FILES,-0.012915,1.000000,-0.009051,0.039553,-0.008580,0.012631,-0.008552,0.146559,-0.003782,0.005658,-0.008022,-0.008392,0.061497,0.018807,0.101296,-0.018134,0.002855,-0.000010,0.038975,0.088949
WRITE_USER_DICTIONARY,0.043064,-0.009051,1.000000,-0.012710,-0.001198,0.141995,-0.007731,0.002126,0.258812,0.033895,-0.007253,0.820356,-0.009716,-0.008644,0.030699,0.013185,-0.000075,0.010712,0.041093,-0.069260
android.intent.action.PACKAGE_CHANGED,0.183920,0.039553,-0.012710,1.000000,0.013431,0.044411,0.300980,0.106075,0.056266,0.104194,0.340281,-0.011785,0.017770,0.180344,0.093653,-0.022713,-0.014408,0.165749,0.079292,-0.076569
android.intent.action.NEW_OUTGOING_CALL,-0.017049,-0.008580,-0.001198,0.013431,1.000000,0.131352,-0.005753,-0.006984,-0.014063,0.109252,0.001604,0.000556,0.773613,0.039618,0.020795,0.017939,0.005850,-0.011187,0.009465,0.085779
REORDER_TASKS,0.009541,0.012631,0.141995,0.044411,0.131352,1.000000,-0.009696,0.075665,-0.005925,0.054078,-0.009096,-0.002384,0.115143,0.058161,0.051036,0.009905,-0.010031,-0.002061,0.194463,-0.073326
WRITE_PROFILE,0.142314,-0.008552,-0.007731,0.300980,-0.005753,-0.009696,1.000000,0.067351,-0.008837,-0.016609,0.701836,-0.007168,-0.008644,0.339626,0.015570,-0.015490,-0.011070,0.434468,-0.013002,-0.065438
SET_WALLPAPER,0.132153,0.146559,0.002126,0.106075,-0.006984,0.075665,0.067351,1.000000,0.003129,0.008307,0.111191,0.005954,-0.019415,0.002154,0.047217,0.112405,0.102106,0.023139,0.039010,0.082225
BIND_INPUT_METHOD,0.057332,-0.003782,0.258812,0.056266,-0.014063,-0.005925,-0.008837,0.003129,1.000000,0.049360,-0.008290,0.350494,-0.012370,-0.003012,-0.006081,0.007203,-0.008287,-0.000544,-0.011356,-0.069736
divideMessage,0.083284,0.005658,0.033895,0.104194,0.109252,0.054078,-0.016609,0.008307,0.049360,1.000000,-0.015581,0.013560,0.112420,0.030206,0.073965,-0.027121,0.069654,0.005760,0.023002,0.082322


In [15]:
data8 = data.iloc[:, 133:152]
data8['class'] = pd.DataFrame(y)
data8.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\173095654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data8['class'] = pd.DataFrame(y)


,ACCESS_COARSE_LOCATION,SET_TIME,android.intent.action.SENDTO,WRITE_SOCIAL_STREAM,WRITE_SETTINGS,REBOOT,BLUETOOTH_ADMIN,TelephonyManager.getNetworkOperator,/system/bin,MessengerService,BIND_DEVICE_ADMIN,WRITE_GSERVICES,IRemoteService,KILL_BACKGROUND_PROCESSES,SET_ALARM,ACCOUNT_MANAGER,/system/app,android.intent.action.CALL,STATUS_BAR,class
ACCESS_COARSE_LOCATION,1.000000,0.007802,0.080137,0.105679,0.068255,0.028439,0.068865,0.275880,0.088582,0.023501,0.087065,-0.016108,0.033164,0.048024,0.035006,-0.024028,0.138896,0.061270,0.037020,0.070071
SET_TIME,0.007802,1.000000,-0.010503,-0.005304,0.129750,0.326362,0.010356,0.007336,-0.010537,-0.004774,-0.001803,0.335711,-0.007152,-0.006538,0.021706,0.176228,-0.007447,0.071867,0.206928,-0.055902
android.intent.action.SENDTO,0.080137,-0.010503,1.000000,0.073692,0.065741,-0.016817,0.028888,0.037513,-0.023518,0.045244,0.084398,-0.009107,-0.013209,0.025582,-0.006969,0.122779,-0.011924,0.180765,-0.004165,-0.063252
WRITE_SOCIAL_STREAM,0.105679,-0.005304,0.073692,1.000000,0.161886,0.183190,0.129244,0.033141,0.017417,-0.004453,0.213045,-0.004600,-0.006671,0.091415,-0.011343,-0.005177,-0.022003,0.119021,-0.006658,-0.053855
WRITE_SETTINGS,0.068255,0.129750,0.065741,0.161886,1.000000,0.198906,0.333111,0.013293,0.032973,0.040069,0.171648,0.079453,0.055394,0.234468,0.295220,0.056518,-0.014779,0.106287,0.243914,-0.065397
REBOOT,0.028439,0.326362,-0.016817,0.183190,0.198906,1.000000,0.145647,0.004639,0.022519,0.018953,0.129584,0.206735,0.000442,0.071392,-0.005246,-0.001252,-0.017219,0.121166,0.147320,-0.061180
BLUETOOTH_ADMIN,0.068865,0.010356,0.028888,0.129244,0.333111,0.145647,1.000000,0.078024,0.115165,0.023244,0.171291,-0.010692,-0.015807,0.305735,0.059593,0.015555,0.059531,0.057619,0.093818,-0.064167
TelephonyManager.getNetworkOperator,0.275880,0.007336,0.037513,0.033141,0.013293,0.004639,0.078024,1.000000,0.116101,0.061562,0.078970,0.010692,0.063141,0.074325,-0.053530,-0.010813,0.185099,0.014237,0.011421,-0.065347
/system/bin,0.088582,-0.010537,-0.023518,0.017417,0.032973,0.022519,0.115165,0.116101,1.000000,0.013314,0.045146,-0.026716,0.072439,0.144920,-0.019945,-0.014499,0.438705,-0.022840,0.006163,0.062762
MessengerService,0.023501,-0.004774,0.045244,-0.004453,0.040069,0.018953,0.023244,0.061562,0.013314,1.000000,0.071357,0.012065,0.139920,0.002998,0.036781,0.009752,0.011152,-0.004392,0.032578,-0.048471


In [16]:
data9 = data.iloc[:, 152:171]
data9['class'] = pd.DataFrame(y)
data9.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\3238528974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data9['class'] = pd.DataFrame(y)


,TelephonyManager.getSimOperator,PERSISTENT_ACTIVITY,CHANGE_NETWORK_STATE,onBind,Process.start,android.intent.action.SCREEN_ON,Context.bindService,RECEIVE_MMS,SET_TIME_ZONE,android.intent.action.BATTERY_OKAY,CONTROL_LOCATION_UPDATES,BROADCAST_WAP_PUSH,BIND_ACCESSIBILITY_SERVICE,ADD_VOICEMAIL,CALL_PHONE,ProcessBuilder,BIND_APPWIDGET,FLASHLIGHT,READ_LOGS,class
TelephonyManager.getSimOperator,1.000000,0.038369,0.175486,0.214296,0.038224,0.044738,-0.014857,0.174734,-0.010168,-0.029970,0.030916,0.065923,0.037080,-0.006665,0.127689,0.157474,0.019520,0.007382,0.073191,-0.052098
PERSISTENT_ACTIVITY,0.038369,1.000000,0.166180,0.046380,0.009182,-0.000515,0.009902,0.009620,0.140225,-0.006851,0.004190,0.037885,0.036604,-0.004112,0.134339,0.004942,0.271955,0.048369,0.149511,-0.047720
CHANGE_NETWORK_STATE,0.175486,0.166180,1.000000,0.124613,-0.007688,0.041514,-0.015936,0.245218,0.049441,-0.011107,-0.018612,0.155358,0.048950,0.016929,0.163792,0.087841,0.024949,0.028279,0.151761,-0.050750
onBind,0.214296,0.046380,0.124613,1.000000,0.032580,0.056091,0.025441,0.050738,0.027042,0.044532,-0.027916,0.038687,0.000437,0.017355,0.072852,0.117107,0.033591,0.011624,0.134250,-0.050314
Process.start,0.038224,0.009182,-0.007688,0.032580,1.000000,-0.004828,0.073572,0.008597,-0.002898,-0.003897,-0.003628,0.011613,-0.002768,-0.002339,-0.010777,0.060323,-0.003447,0.007234,0.017933,-0.040042
android.intent.action.SCREEN_ON,0.044738,-0.000515,0.041514,0.056091,-0.004828,1.000000,-0.004647,0.033336,0.008045,0.042164,0.004123,-0.007478,-0.004886,-0.004128,0.051332,-0.015834,-0.006084,0.002843,0.035212,-0.045078
Context.bindService,-0.014857,0.009902,-0.015936,0.025441,0.073572,-0.004647,1.000000,0.000848,-0.002789,-0.003751,-0.003493,0.012383,-0.002665,-0.002251,-0.009765,0.003077,-0.003318,0.041136,-0.014694,-0.038546
RECEIVE_MMS,0.174734,0.009620,0.245218,0.050738,0.008597,0.033336,0.000848,1.000000,-0.000732,0.023753,0.115738,0.258797,0.057959,-0.006975,0.232246,0.037951,0.003037,-0.012167,0.061686,0.048936
SET_TIME_ZONE,-0.010168,0.140225,0.049441,0.027042,-0.002898,0.008045,-0.002789,-0.000732,1.000000,-0.004129,0.013601,0.100271,-0.002933,0.159446,0.055782,-0.017205,0.014704,0.020822,0.010697,-0.039926
android.intent.action.BATTERY_OKAY,-0.029970,-0.006851,-0.011107,0.044532,-0.003897,0.042164,-0.003751,0.023753,-0.004129,1.000000,0.020836,-0.006037,-0.003944,-0.003332,0.139660,-0.013630,-0.004911,-0.006720,0.001616,0.047105


In [17]:
data10 = data.iloc[:, 171:190]
data10['class'] = pd.DataFrame(y)
data10.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\2121588230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data10['class'] = pd.DataFrame(y)


,Ljava.lang.Class.getResource,defineClass,SET_PROCESS_LIMIT,android.intent.action.PACKAGE_RESTARTED,MOUNT_UNMOUNT_FILESYSTEMS,BIND_TEXT_SERVICE,INSTALL_LOCATION_PROVIDER,android.intent.action.CALL_BUTTON,android.intent.action.SCREEN_OFF,findClass,SYSTEM_ALERT_WINDOW,MOUNT_FORMAT_FILESYSTEMS,CHANGE_CONFIGURATION,CLEAR_APP_USER_DATA,intent.action.RUN,android.intent.action.SET_WALLPAPER,CHANGE_WIFI_STATE,READ_FRAME_BUFFER,ACCESS_SURFACE_FLINGER,class
Ljava.lang.Class.getResource,1.000000,0.132345,-0.002085,-0.030762,-0.009916,-0.023997,-0.017909,0.013090,0.016561,0.361979,-0.008236,-0.001090,-0.049879,-0.032774,0.030487,-0.033542,0.078765,0.007573,-0.027046,0.042149
defineClass,0.132345,1.000000,-0.005193,-0.009763,-0.017169,-0.004334,-0.005033,0.019052,0.053744,0.182148,0.006107,-0.005719,-0.002591,-0.005999,-0.005999,-0.014221,0.002524,-0.010220,-0.005930,-0.038137
SET_PROCESS_LIMIT,-0.002085,-0.005193,1.000000,0.158296,0.015046,-0.001836,0.342519,-0.002330,-0.004082,-0.000893,0.046699,0.025168,0.082247,0.023769,-0.002542,-0.006026,0.095137,0.026729,0.024101,-0.032970
android.intent.action.PACKAGE_RESTARTED,-0.030762,-0.009763,0.158296,1.000000,0.068332,-0.003452,0.146812,-0.004381,0.027530,-0.010807,0.112073,0.128259,0.028939,-0.004778,0.009294,-0.011328,0.058230,0.008471,0.009511,-0.036021
MOUNT_UNMOUNT_FILESYSTEMS,-0.009916,-0.017169,0.015046,0.068332,1.000000,0.011695,-0.000272,-0.001745,0.001002,-0.015516,0.075509,0.278182,0.082324,0.147729,-0.010061,-0.012121,0.058225,-0.004993,0.017812,0.038274
BIND_TEXT_SERVICE,-0.023997,-0.004334,-0.001836,-0.003452,0.011695,1.000000,-0.001780,-0.001945,-0.003407,-0.012796,0.002016,-0.002022,-0.006356,-0.002121,-0.002121,-0.005029,-0.017208,-0.003614,-0.002097,-0.029973
INSTALL_LOCATION_PROVIDER,-0.017909,-0.005033,0.342519,0.146812,-0.000272,-0.001780,1.000000,-0.002258,-0.003957,-0.014860,0.049004,0.026117,0.085329,0.024681,-0.002463,-0.005840,0.071575,0.027847,0.025022,-0.031769
android.intent.action.CALL_BUTTON,0.013090,0.019052,-0.002330,-0.004381,-0.001745,-0.001945,-0.002258,1.000000,-0.004323,-0.007145,0.083750,-0.002566,0.000423,-0.002692,-0.002692,-0.006381,0.000031,-0.004586,-0.002661,-0.032471
android.intent.action.SCREEN_OFF,0.016561,0.053744,-0.004082,0.027530,0.001002,-0.003407,-0.003957,-0.004323,1.000000,0.000239,0.016801,0.025405,-0.014130,-0.004716,0.066566,-0.011180,0.024465,0.000381,0.038600,-0.034724
findClass,0.361979,0.182148,-0.000893,-0.010807,-0.015516,-0.012796,-0.014860,-0.007145,0.000239,1.000000,-0.035969,-0.003767,-0.033129,-0.017711,0.015647,-0.013467,-0.057460,0.053507,-0.004855,0.036756


In [18]:
data11 = data.iloc[:, 190:209]
data11['class'] = pd.DataFrame(y)
data11.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\3396703351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data11['class'] = pd.DataFrame(y)


,Runtime.loadLibrary,BROADCAST_SMS,EXPAND_STATUS_BAR,INTERNAL_SYSTEM_WINDOW,android.intent.action.BATTERY_LOW,SET_ACTIVITY_WATCHER,WRITE_CONTACTS,android.intent.action.ACTION_POWER_CONNECTED,BIND_VPN_SERVICE,DISABLE_KEYGUARD,ACCESS_MOCK_LOCATION,GET_PACKAGE_SIZE,MODIFY_PHONE_STATE,CHANGE_COMPONENT_ENABLED_STATE,CLEAR_APP_CACHE,SET_ORIENTATION,READ_CONTACTS,DEVICE_POWER,HARDWARE_TEST,class
Runtime.loadLibrary,1.000000,-0.003108,0.010147,-0.001984,0.022516,0.063585,0.064097,-0.007986,-0.000729,0.022476,-0.003670,0.011739,-0.004495,-0.003098,0.013938,0.022516,0.037357,-0.004210,-0.002067,-0.024199
BROADCAST_SMS,-0.003108,1.000000,0.023922,0.201361,0.065169,0.017734,0.176021,0.047043,-0.002270,0.122703,0.077991,0.023060,0.221187,0.025540,0.028416,-0.000127,0.118507,0.112154,0.004058,-0.028608
EXPAND_STATUS_BAR,0.010147,0.023922,1.000000,0.043004,-0.012290,0.009527,0.098874,0.069085,-0.003365,0.013779,0.064390,0.125309,0.019367,0.033704,0.088375,0.004414,0.153052,0.048219,0.004775,-0.027679
INTERNAL_SYSTEM_WINDOW,-0.001984,0.201361,0.043004,1.000000,-0.005290,0.030579,0.012730,-0.015861,-0.001449,0.069645,0.029854,0.007714,0.113193,0.114415,0.001399,0.032854,0.035795,0.211152,0.012236,-0.026021
android.intent.action.BATTERY_LOW,0.022516,0.065169,-0.012290,-0.005290,1.000000,0.021630,0.138138,0.229768,-0.001945,0.172432,0.024907,0.103259,0.067860,-0.008259,-0.003456,0.002399,0.015528,0.000926,-0.005511,0.027714
SET_ACTIVITY_WATCHER,0.063585,0.017734,0.009527,0.030579,0.021630,1.000000,0.021732,0.000342,-0.000753,0.090034,0.014015,0.026567,0.009993,0.017818,0.030432,0.046011,0.006106,0.026832,0.029199,-0.020767
WRITE_CONTACTS,0.064097,0.176021,0.098874,0.012730,0.138138,0.021732,1.000000,0.372344,-0.007981,0.300663,0.025620,0.082617,0.184092,0.010474,0.076228,-0.018854,0.572887,0.064190,0.010458,-0.022991
android.intent.action.ACTION_POWER_CONNECTED,-0.007986,0.047043,0.069085,-0.015861,0.229768,0.000342,0.372344,1.000000,-0.005831,0.352447,-0.019562,0.048945,0.090660,-0.010334,-0.016282,-0.021293,0.193697,-0.007968,-0.016522,-0.022257
BIND_VPN_SERVICE,-0.000729,-0.002270,-0.003365,-0.001449,-0.001945,-0.000753,-0.007981,-0.005831,1.000000,0.014968,-0.002680,-0.003097,-0.003282,0.057169,-0.002780,-0.001945,0.000906,-0.003074,-0.001509,-0.017668
DISABLE_KEYGUARD,0.022476,0.122703,0.013779,0.069645,0.172432,0.090034,0.300663,0.352447,0.014968,1.000000,0.036128,0.122067,0.191579,-0.007754,0.143797,-0.015404,0.204522,0.133098,0.023256,0.021212


In [19]:
data12 = data.iloc[:, 209:215]
data12['class'] = pd.DataFrame(y)
data12.corr()

C:\Users\masterzrafly\AppData\Local\Temp\ipykernel_9968\3184774273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data12['class'] = pd.DataFrame(y)


,ACCESS_WIFI_STATE,WRITE_EXTERNAL_STORAGE,ACCESS_FINE_LOCATION,SET_WALLPAPER_HINTS,SET_PREFERRED_APPLICATIONS,WRITE_SECURE_SETTINGS,class
ACCESS_WIFI_STATE,1.000000,0.279336,0.215900,0.009904,0.039994,0.095347,0.004997
WRITE_EXTERNAL_STORAGE,0.279336,1.000000,0.248221,0.047639,0.003990,0.037815,0.007422
ACCESS_FINE_LOCATION,0.215900,0.248221,1.000000,0.025522,0.040543,0.053949,0.017259
SET_WALLPAPER_HINTS,0.009904,0.047639,0.025522,1.000000,0.058869,-0.020034,-0.015435
SET_PREFERRED_APPLICATIONS,0.039994,0.003990,0.040543,0.058869,1.000000,0.085290,0.011243
WRITE_SECURE_SETTINGS,0.095347,0.037815,0.053949,-0.020034,0.085290,1.000000,-0.005872
class,0.004997,0.007422,0.017259,-0.015435,0.011243,-0.005872,1.000000


# Spliting Data

In [20]:
data['class'] = pd.DataFrame(y)
X = data.drop(columns=['class'])
Y = data['class']
xAll_train, xAll_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
dataPositif = data[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'android.telephony.gsm.SmsManager', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET', 'createSubprocess', 'ACCESS_LOCATION_EXTRA_COMMANDS', 'WRITE_APN_SETTINGS', 'abortBroadcast', 'TelephonyManager.getDeviceId', 'RECEIVE_BOOT_COMPLETED', 'RESTART_PACKAGES', 'chmod', 'TelephonyManager.getSimSerialNumber', 'PackageInstaller', 'remount', 'sendDataMessage', 'chown', 'UPDATE_DEVICE_STATS', 'DELETE_PACKAGES', 'DELETE_CACHE_FILES', 'android.intent.action.NEW_OUTGOING_CALL', 'SET_WALLPAPER', 'divideMessage', 'PROCESS_OUTGOING_CALLS', 'BIND_WALLPAPER', 'RECEIVE_WAP_PUSH', 'ACCESS_COARSE_LOCATION', '/system/bin', 'SET_ALARM', 'RECEIVE_MMS', 'android.intent.action.BATTERY_OKAY', 'CONTROL_LOCATION_UPDATES', 'CALL_PHONE', 'READ_LOGS', 'Ljava.lang.Class.getResource', 'MOUNT_UNMOUNT_FILESYSTEMS', 'findClass', 'CHANGE_CONFIGURATION', 'CHANGE_WIFI_STATE', 'android.intent.action.BATTERY_LOW', 'DISABLE_KEYGUARD', 'MODIFY_PHONE_STATE', 'CLEAR_APP_CACHE', 'READ_CONTACTS', 'HARDWARE_TEST', 'ACCESS_WIFI_STATE', 'WRITE_EXTERNAL_STORAGE', 'ACCESS_FINE_LOCATION', 'SET_PREFERRED_APPLICATIONS']]
xPositif_train = xAll_train[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'android.telephony.gsm.SmsManager', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET', 'createSubprocess', 'ACCESS_LOCATION_EXTRA_COMMANDS', 'WRITE_APN_SETTINGS', 'abortBroadcast', 'TelephonyManager.getDeviceId', 'RECEIVE_BOOT_COMPLETED', 'RESTART_PACKAGES', 'chmod', 'TelephonyManager.getSimSerialNumber', 'PackageInstaller', 'remount', 'sendDataMessage', 'chown', 'UPDATE_DEVICE_STATS', 'DELETE_PACKAGES', 'DELETE_CACHE_FILES', 'android.intent.action.NEW_OUTGOING_CALL', 'SET_WALLPAPER', 'divideMessage', 'PROCESS_OUTGOING_CALLS', 'BIND_WALLPAPER', 'RECEIVE_WAP_PUSH', 'ACCESS_COARSE_LOCATION', '/system/bin', 'SET_ALARM', 'RECEIVE_MMS', 'android.intent.action.BATTERY_OKAY', 'CONTROL_LOCATION_UPDATES', 'CALL_PHONE', 'READ_LOGS', 'Ljava.lang.Class.getResource', 'MOUNT_UNMOUNT_FILESYSTEMS', 'findClass', 'CHANGE_CONFIGURATION', 'CHANGE_WIFI_STATE', 'android.intent.action.BATTERY_LOW', 'DISABLE_KEYGUARD', 'MODIFY_PHONE_STATE', 'CLEAR_APP_CACHE', 'READ_CONTACTS', 'HARDWARE_TEST', 'ACCESS_WIFI_STATE', 'WRITE_EXTERNAL_STORAGE', 'ACCESS_FINE_LOCATION', 'SET_PREFERRED_APPLICATIONS']]
xPositif_test =  xAll_test[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'android.telephony.gsm.SmsManager', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET', 'createSubprocess', 'ACCESS_LOCATION_EXTRA_COMMANDS', 'WRITE_APN_SETTINGS', 'abortBroadcast', 'TelephonyManager.getDeviceId', 'RECEIVE_BOOT_COMPLETED', 'RESTART_PACKAGES', 'chmod', 'TelephonyManager.getSimSerialNumber', 'PackageInstaller', 'remount', 'sendDataMessage', 'chown', 'UPDATE_DEVICE_STATS', 'DELETE_PACKAGES', 'DELETE_CACHE_FILES', 'android.intent.action.NEW_OUTGOING_CALL', 'SET_WALLPAPER', 'divideMessage', 'PROCESS_OUTGOING_CALLS', 'BIND_WALLPAPER', 'RECEIVE_WAP_PUSH', 'ACCESS_COARSE_LOCATION', '/system/bin', 'SET_ALARM', 'RECEIVE_MMS', 'android.intent.action.BATTERY_OKAY', 'CONTROL_LOCATION_UPDATES', 'CALL_PHONE', 'READ_LOGS', 'Ljava.lang.Class.getResource', 'MOUNT_UNMOUNT_FILESYSTEMS', 'findClass', 'CHANGE_CONFIGURATION', 'CHANGE_WIFI_STATE', 'android.intent.action.BATTERY_LOW', 'DISABLE_KEYGUARD', 'MODIFY_PHONE_STATE', 'CLEAR_APP_CACHE', 'READ_CONTACTS', 'HARDWARE_TEST', 'ACCESS_WIFI_STATE', 'WRITE_EXTERNAL_STORAGE', 'ACCESS_FINE_LOCATION', 'SET_PREFERRED_APPLICATIONS']]

In [22]:
data04 = data[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE']]
x04_train = xAll_train[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE']]
x04_test = xAll_test[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE']]

In [23]:
data03 = data[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number']]
x03_train = xAll_train[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number']]
x03_test = xAll_test[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number']]

In [24]:
data02 = data[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'android.telephony.gsm.SmsManager', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET']]
x02_train = xAll_train[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'android.telephony.gsm.SmsManager', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET']]
x02_test = xAll_test[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'android.telephony.gsm.SmsManager', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET']]

In [25]:
data01 = data[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'android.telephony.gsm.SmsManager', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET', 'ACCESS_LOCATION_EXTRA_COMMANDS', 'WRITE_APN_SETTINGS', 'abortBroadcast', 'createSubprocess', 'TelephonyManager.getDeviceId', 'RECEIVE_BOOT_COMPLETED', 'RESTART_PACKAGES', 'chmod', 'TelephonyManager.getSimSerialNumber', 'PackageInstaller', 'remount', 'sendDataMessage', 'chown']]
x01_train = xAll_train[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'android.telephony.gsm.SmsManager', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET', 'ACCESS_LOCATION_EXTRA_COMMANDS', 'WRITE_APN_SETTINGS', 'abortBroadcast', 'createSubprocess', 'TelephonyManager.getDeviceId', 'RECEIVE_BOOT_COMPLETED', 'RESTART_PACKAGES', 'chmod', 'TelephonyManager.getSimSerialNumber', 'PackageInstaller', 'remount', 'sendDataMessage', 'chown']]
x01_test = xAll_test[['SEND_SMS', 'android.telephony.SmsManager', 'READ_PHONE_STATE', 'RECEIVE_SMS', 'READ_SMS', 'android.intent.action.BOOT_COMPLETED', 'TelephonyManager.getLine1Number', 'WRITE_SMS', 'WRITE_HISTORY_BOOKMARKS', 'TelephonyManager.getSubscriberId', 'android.telephony.gsm.SmsManager', 'INSTALL_PACKAGES', 'READ_HISTORY_BOOKMARKS', 'INTERNET', 'ACCESS_LOCATION_EXTRA_COMMANDS', 'WRITE_APN_SETTINGS', 'abortBroadcast', 'createSubprocess', 'TelephonyManager.getDeviceId', 'RECEIVE_BOOT_COMPLETED', 'RESTART_PACKAGES', 'chmod', 'TelephonyManager.getSimSerialNumber', 'PackageInstaller', 'remount', 'sendDataMessage', 'chown']]

# Pembagian data menjadi 4 bagian untuk PSVM SMO

In [26]:
def splitting_Data(data):
    data = data.sample(frac = 1)
    n = len(data)
    # data 1 - n/4
    c1 = data.iloc[:int(n/4),:]
    # data n/4 - n/2
    c2 = data.iloc[int(n/4):int(n/2),:]
    # data n/2 - 3n/4
    c3 = data.iloc[int(n/2):int(3*n/4),:]
    # data 3n/4 - n
    c4 = data.iloc[int(3*n/4):n,:]
    return c1, c2, c3, c4

# SMO

In [27]:
class SVM():
    def __init__(self, max_iter=100, kernel_type='linear', C=1.0, epsilon=0.001):
        self.kernels = {
            'linear' : self.kernel_linear,
            'quadratic' : self.kernel_quadratic,
            'gaussian' : self.kernel_gauss
        }
        self.max_iter = max_iter
        self.kernel_type = kernel_type
        self.C = C
        self.epsilon = epsilon
    def fit(self, X, y):
        n, d = X.shape[0], X.shape[1]
        alpha = np.zeros((n))
        kernel = self.kernels[self.kernel_type]
        count = 0
        while True:
            count += 1
            alpha_prev = np.copy(alpha)
            for j in range(0, n):
                i = self.get_rnd_int(0, n-1, j) # Get random int i~=j
                x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]
                k_ij = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                if k_ij == 0:
                    continue
                alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
                (L, H) = self.compute_L_H(self.C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                # Compute model parameters
                self.w = self.calc_w(alpha, y, X)
                self.b = self.calc_b(X, y, self.w)

                # Compute E_i, E_j
                E_i = self.E(x_i, y_i, self.w, self.b)
                E_j = self.E(x_j, y_j, self.w, self.b)

                # Set new alpha values
                alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)

                alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])
            # Check convergence
            diff = np.linalg.norm(alpha - alpha_prev)
            if diff < self.epsilon:
                break
            #print(count)
            if count >= self.max_iter:
                print("Iteration number exceeded the max of %d iterations" % (self.max_iter))
                return
        self.b = self.calc_b(X, y, self.w)
        if self.kernel_type == 'linear':
            self.w = self.calc_w(alpha, y, X)
        # Get support vectors
        alpha_idx = np.where(alpha > 0)[0]
        support_vectors = X[alpha_idx, :]
        return support_vectors, count
    def predict(self, X):
        return self.h(X, self.w, self.b)
    def bobotW(self):
        return self.w
    def bobotB(self):
        return self.b
    def calc_b(self, X, y, w):
        b_tmp = y - np.dot(w.T, X.T)
        return np.mean(b_tmp)
    def calc_w(self, alpha, y, X):
        return np.dot(X.T, np.multiply(alpha,y))
    def h(self, X, w, b):
        return np.sign(np.dot(w.T, X.T) + b).astype(int)
    def E(self, x_k, y_k, w, b):
        return self.h(x_k, w, b) - y_k
    def compute_L_H(self, C, alpha_prime_j, alpha_prime_i, y_j, y_i):
        if(y_i != y_j):
            return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
        else:
            return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))
    def get_rnd_int(self, a,b,z):
        i = z
        cnt=0
        while i == z and cnt<1000:
            i = rnd.randint(a,b)
            cnt=cnt+1
        return i
    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)
    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)
    def kernel_gauss(self,x1, x2, sigma=1):
        return np.exp(- (np.linalg.norm(x1 - x2, 2)) ** 2 / (2 * sigma ** 2))
    def predict_proba(self, X):
        return np.dot(self.w.T, X.T) + self.b

# Cor > 0.1

In [28]:
start = time.time()

model = SVM(max_iter=3, C=0.1)
model.fit(x01_train.to_numpy(), y_train.to_numpy())
Y_predicted = [model.predict(x) for x in data01.to_numpy()]

for i in range(len(Y_predicted)):
  if(Y_predicted[i]==-1):
    Y_predicted[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted))
print(classification_report(y, Y_predicted))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted)*100, 2), "%")
print('Nilai W : ', model.bobotW())
print('Nilai B : ', model.bobotB())

Iteration number exceeded the max of 3 iterations
time :  158.25544667243958
[[6673 2803]
 [1018 4537]]
              precision    recall  f1-score   support

           0       0.87      0.70      0.78      9476
           1       0.62      0.82      0.70      5555

    accuracy                           0.75     15031
   macro avg       0.74      0.76      0.74     15031
weighted avg       0.78      0.75      0.75     15031

Akurasi Model :  74.58 %
Nilai W :  [121.76829641  92.03729408 187.84029206  79.73076727  77.93156647
 129.61944624  80.72510823  43.32534035  34.77316017  66.36217018
  26.7274549   30.80707626  38.11674992 216.13512958  19.63523046
  19.75670898  40.89758379  11.38634699 136.65767468  97.78006895
  27.43279955  36.34284049  33.83727939  12.7972028   11.24275724
   7.58747821   7.71748252]
Nilai B :  -620.187134574133


In [29]:
start = time.time()

model1 = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
model1.fit(x01_train.to_numpy(), y_train.to_numpy())
Y_predicted1 = [model1.predict(x) for x in data01.to_numpy()]

for i in range(len(Y_predicted1)):
  if(Y_predicted1[i]==-1):
    Y_predicted1[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted1))
print(classification_report(y, Y_predicted1))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted1)*100, 2), "%")
print('Nilai W : ', model1.bobotW())
print('Nilai B : ', model1.bobotB())

Iteration number exceeded the max of 3 iterations
time :  155.78054428100586
[[6681 2795]
 [1024 4531]]
              precision    recall  f1-score   support

           0       0.87      0.71      0.78      9476
           1       0.62      0.82      0.70      5555

    accuracy                           0.75     15031
   macro avg       0.74      0.76      0.74     15031
weighted avg       0.78      0.75      0.75     15031

Akurasi Model :  74.59 %
Nilai W :  [120.5  90.9 191.6  76.8  76.9 131.3  82.2  44.   33.5  68.1  25.7  32.8
  37.4 219.6  24.3  19.7  38.3  11.9 138.1  97.7  26.3  39.6  35.3  13.8
  11.6   7.6   8.3]
Nilai B :  -627.0311543579512


# Cor > 0.2

In [30]:
start = time.time()

model = SVM(max_iter=3, C=0.1)
model.fit(x02_train.to_numpy(), y_train.to_numpy())
Y_predicted = [model.predict(x) for x in data02.to_numpy()]

for i in range(len(Y_predicted)):
  if(Y_predicted[i]==-1):
    Y_predicted[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted))
print(classification_report(y, Y_predicted))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted)*100, 2), "%")
print('Nilai W : ', model.bobotW())
print('Nilai B : ', model.bobotB())

Iteration number exceeded the max of 3 iterations
time :  86.22648453712463
[[7017 2459]
 [ 801 4754]]
              precision    recall  f1-score   support

           0       0.90      0.74      0.81      9476
           1       0.66      0.86      0.74      5555

    accuracy                           0.78     15031
   macro avg       0.78      0.80      0.78     15031
weighted avg       0.81      0.78      0.79     15031

Akurasi Model :  78.31 %
Nilai W :  [110.8         85.         183.09090909  75.1         75.3
 126.69090909  80.39090909  43.7         35.7         68.
  24.9         30.2         39.3        207.69090909]
Nilai B :  -473.6391081473421


In [31]:
start = time.time()

model1 = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
model1.fit(x02_train.to_numpy(), y_train.to_numpy())
Y_predicted1 = [model1.predict(x) for x in data02.to_numpy()]

for i in range(len(Y_predicted1)):
  if(Y_predicted1[i]==-1):
    Y_predicted1[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted1))
print(classification_report(y, Y_predicted1))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted1)*100, 2), "%")
print('Nilai W : ', model1.bobotW())
print('Nilai B : ', model1.bobotB())

Iteration number exceeded the max of 3 iterations
time :  86.7981550693512
[[7016 2460]
 [ 801 4754]]
              precision    recall  f1-score   support

           0       0.90      0.74      0.81      9476
           1       0.66      0.86      0.74      5555

    accuracy                           0.78     15031
   macro avg       0.78      0.80      0.78     15031
weighted avg       0.81      0.78      0.79     15031

Akurasi Model :  78.3 %
Nilai W :  [107.6  79.7 174.1  73.9  71.8 119.7  75.8  41.5  33.   65.1  24.4  27.6
  36.6 197.3]
Nilai B :  -450.4170908183635


# Cor > 0.3

In [32]:
start = time.time()

model = SVM(max_iter=3, C=0.1)
model.fit(x03_train.to_numpy(), y_train.to_numpy())
Y_predicted = [model.predict(x) for x in data03.to_numpy()]

for i in range(len(Y_predicted)):
  if(Y_predicted[i]==-1):
    Y_predicted[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted))
print(classification_report(y, Y_predicted))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted)*100, 2), "%")
print('Nilai W : ', model.bobotW())
print('Nilai B : ', model.bobotB())

Iteration number exceeded the max of 3 iterations
time :  17.251091957092285
[[6759 2717]
 [ 745 4810]]
              precision    recall  f1-score   support

           0       0.90      0.71      0.80      9476
           1       0.64      0.87      0.74      5555

    accuracy                           0.77     15031
   macro avg       0.77      0.79      0.77     15031
weighted avg       0.80      0.77      0.77     15031

Akurasi Model :  76.97 %
Nilai W :  [109.2  81.8 175.4  73.9  73.  117.2  77.2]
Nilai B :  -251.5452677977377


In [33]:
start = time.time()

model1 = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
model1.fit(x03_train.to_numpy(), y_train.to_numpy())
Y_predicted1 = [model1.predict(x) for x in data03.to_numpy()]

for i in range(len(Y_predicted1)):
  if(Y_predicted1[i]==-1):
    Y_predicted1[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted1))
print(classification_report(y, Y_predicted1))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted1)*100, 2), "%")
print('Nilai W : ', model1.bobotW())
print('Nilai B : ', model1.bobotB())

Iteration number exceeded the max of 3 iterations
time :  19.053056240081787
[[6759 2717]
 [ 745 4810]]
              precision    recall  f1-score   support

           0       0.90      0.71      0.80      9476
           1       0.64      0.87      0.74      5555

    accuracy                           0.77     15031
   macro avg       0.77      0.79      0.77     15031
weighted avg       0.80      0.77      0.77     15031

Akurasi Model :  76.97 %
Nilai W :  [109.3  82.3 176.9  73.8  72.8 119.7  77.6]
Nilai B :  -253.78036427145693


# Cor > 0.4

In [34]:
start = time.time()

model = SVM(max_iter=3, C=0.1)
model.fit(x04_train.to_numpy(), y_train.to_numpy())
Y_predicted = [model.predict(x) for x in data04.to_numpy()]

for i in range(len(Y_predicted)):
  if(Y_predicted[i]==-1):
    Y_predicted[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted))
print(classification_report(y, Y_predicted))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted)*100, 2), "%")
print('Nilai W : ', model.bobotW())
print('Nilai B : ', model.bobotB())

Iteration number exceeded the max of 3 iterations
time :  9.69343113899231
[[4887 4589]
 [ 378 5177]]
              precision    recall  f1-score   support

           0       0.93      0.52      0.66      9476
           1       0.53      0.93      0.68      5555

    accuracy                           0.67     15031
   macro avg       0.73      0.72      0.67     15031
weighted avg       0.78      0.67      0.67     15031

Akurasi Model :  66.95 %
Nilai W :  [152.9 110.5 160.5]
Nilai B :  -158.06860445775115


In [35]:
start = time.time()

model1 = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
model1.fit(x04_train.to_numpy(), y_train.to_numpy())
Y_predicted1 = [model1.predict(x) for x in data04.to_numpy()]

for i in range(len(Y_predicted1)):
  if(Y_predicted1[i]==-1):
    Y_predicted1[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted1))
print(classification_report(y, Y_predicted1))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted1)*100, 2), "%")
print('Nilai W : ', model1.bobotW())
print('Nilai B : ', model1.bobotB())

Iteration number exceeded the max of 3 iterations
time :  11.033665418624878
[[4887 4589]
 [ 378 5177]]
              precision    recall  f1-score   support

           0       0.93      0.52      0.66      9476
           1       0.53      0.93      0.68      5555

    accuracy                           0.67     15031
   macro avg       0.73      0.72      0.67     15031
weighted avg       0.78      0.67      0.67     15031

Akurasi Model :  66.95 %
Nilai W :  [152.  110.4 160.6]
Nilai B :  -157.90011643379907


# All Data Corr Positif

In [36]:
start = time.time()

model = SVM(max_iter=3, C=0.1)
model.fit(xPositif_train.to_numpy(), y_train.to_numpy())
Y_predicted = [model.predict(x) for x in dataPositif.to_numpy()]

for i in range(len(Y_predicted)):
  if(Y_predicted[i]==-1):
    Y_predicted[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted))
print(classification_report(y, Y_predicted))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted)*100, 2), "%")
print('Nilai W : ', model.bobotW())
print('Nilai B : ', model.bobotB())

Iteration number exceeded the max of 3 iterations
time :  328.0556130409241
[[6106 3370]
 [1328 4227]]
              precision    recall  f1-score   support

           0       0.82      0.64      0.72      9476
           1       0.56      0.76      0.64      5555

    accuracy                           0.69     15031
   macro avg       0.69      0.70      0.68     15031
weighted avg       0.72      0.69      0.69     15031

Akurasi Model :  68.74 %
Nilai W :  [132.83668905 103.02093978 206.59438505  84.79713021  75.72425896
 133.75998388  89.37050854  41.54636478  25.56758335  33.8071037
  67.51553336  28.9407802   37.82018923 234.87577212  12.34180554
  21.85478813  16.97090843  36.3791619  139.32895772 100.69572113
  28.08429567  38.6438461   34.17222943  10.75153251  11.33321614
   7.61986964   7.98000518   6.08216732   9.29658452   3.83554166
   4.94275384  20.49585263  11.10778702   6.28867989  11.84149792
   7.33778348  66.25514342  37.28715307   7.99027172   5.55408172
   2.01

In [37]:
start = time.time()

model1 = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
model1.fit(xPositif_train.to_numpy(), y_train.to_numpy())
Y_predicted1 = [model1.predict(x) for x in dataPositif.to_numpy()]

for i in range(len(Y_predicted1)):
  if(Y_predicted1[i]==-1):
    Y_predicted1[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted1))
print(classification_report(y, Y_predicted1))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted1)*100, 2), "%")
print('Nilai W : ', model1.bobotW())
print('Nilai B : ', model1.bobotB())

Iteration number exceeded the max of 3 iterations
time :  332.0893003940582
[[6193 3283]
 [1355 4200]]
              precision    recall  f1-score   support

           0       0.82      0.65      0.73      9476
           1       0.56      0.76      0.64      5555

    accuracy                           0.69     15031
   macro avg       0.69      0.70      0.69     15031
weighted avg       0.72      0.69      0.70     15031

Akurasi Model :  69.14 %
Nilai W :  [148.7 113.1 229.7  98.5  90.1 151.7 103.5  52.1  30.9  38.6  83.8  35.2
  42.6 258.6  15.   24.3  23.2  46.4 161.  116.4  35.1  45.   42.   13.2
  14.2   9.4  10.    7.5  10.6   5.2   7.9  23.8  13.6   9.6  12.9   9.1
  75.   44.8   8.5   7.9   2.2   2.5  33.   23.7 101.6   9.9  27.9   7.8
  44.    2.2  19.6   5.3   3.   56.6   1.6 106.1 161.5  70.2   2. ]
Nilai B :  -1021.6503576180979


# All Data

In [38]:
start = time.time()

model = SVM(max_iter=3, C=0.1)
model.fit(xAll_train.to_numpy(), y_train.to_numpy())
Y_predicted = [model.predict(x) for x in data.drop(columns=['class']).to_numpy()]

for i in range(len(Y_predicted)):
  if(Y_predicted[i]==-1):
    Y_predicted[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted))
print(classification_report(y, Y_predicted))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted)*100, 2), "%")
print('Nilai W : ', model.bobotW())
print('Nilai B : ', model.bobotB())

Iteration number exceeded the max of 3 iterations
time :  1171.0855042934418
[[3779 5697]
 [2540 3015]]
              precision    recall  f1-score   support

           0       0.60      0.40      0.48      9476
           1       0.35      0.54      0.42      5555

    accuracy                           0.45     15031
   macro avg       0.47      0.47      0.45     15031
weighted avg       0.50      0.45      0.46     15031

Akurasi Model :  45.2 %
Nilai W :  [1.01563612e+01 1.27965104e+01 1.23371704e+01 1.00493795e+01
 1.30476436e+01 2.35493418e+01 1.33620842e+02 4.82589176e+00
 4.55084642e+00 7.55601918e+00 1.66775984e+01 9.82359291e+00
 1.05207190e+02 1.86959204e+02 8.02167034e-01 3.69134358e+01
 5.21372978e+01 3.56942008e+01 4.17939404e+01 1.20144544e+01
 8.53966413e+01 3.99949146e+01 7.09032019e+01 1.51219355e-01
 3.06539443e+01 1.14448975e+02 6.52173913e-02 1.29762964e-01
 8.15785300e+01 3.80931983e+01 7.10939512e+01 5.65305878e+00
 5.69692902e+01 3.71350535e+01 6.42562180e+01 

In [39]:
start = time.time()

model1 = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
model1.fit(xAll_train.to_numpy(), y_train.to_numpy())
Y_predicted1 = [model1.predict(x) for x in data.drop(columns=['class']).to_numpy()]

for i in range(len(Y_predicted1)):
  if(Y_predicted1[i]==-1):
    Y_predicted1[i] = 0
    
stop = time.time()
print('time : ',stop-start)

print(confusion_matrix(y, Y_predicted1))
print(classification_report(y, Y_predicted1))
print("Akurasi Model : ", round(accuracy_score(y, Y_predicted1)*100, 2), "%")
print('Nilai W : ', model1.bobotW())
print('Nilai B : ', model1.bobotB())

Iteration number exceeded the max of 3 iterations
time :  1192.3802971839905
[[3731 5745]
 [2878 2677]]
              precision    recall  f1-score   support

           0       0.56      0.39      0.46      9476
           1       0.32      0.48      0.38      5555

    accuracy                           0.43     15031
   macro avg       0.44      0.44      0.42     15031
weighted avg       0.47      0.43      0.43     15031

Akurasi Model :  42.63 %
Nilai W :  [2.000e+01 2.770e+01 2.680e+01 2.000e+01 2.800e+01 4.440e+01 2.106e+02
 1.500e+01 9.900e+00 1.540e+01 4.060e+01 2.800e+01 1.594e+02 3.295e+02
 1.700e+00 8.620e+01 9.170e+01 7.030e+01 6.610e+01 2.900e+01 1.483e+02
 8.160e+01 1.366e+02 4.000e-01 7.230e+01 2.316e+02 2.000e-01 5.000e-01
 1.679e+02 9.060e+01 1.455e+02 1.850e+01 1.153e+02 8.910e+01 1.395e+02
 6.450e+01 4.700e+00 8.490e+01 7.650e+01 2.000e+00 1.000e-01 4.430e+01
 5.460e+01 1.165e+02 1.556e+02 5.280e+01 9.790e+01 1.530e+01 2.077e+02
 2.700e+00 6.000e+01 1.690e+02 3.587

# Parallel SVM Corr > 0.1

In [40]:
dataParallel01 = x01_train.copy()
dataParallel01['class'] = y_train.copy()
c1, c2, c3, c4 = splitting_Data(dataParallel01)

x1 = c1.drop(columns=['class']).to_numpy()
y1 = c1['class'].to_numpy()

x2 = c2.drop(columns=['class']).to_numpy()
y2 = c2['class'].to_numpy()

x3 = c3.drop(columns=['class']).to_numpy()
y3 = c3['class'].to_numpy()

x4 = c4.drop(columns=['class']).to_numpy()
y4 = c4['class'].to_numpy()

In [41]:
def PSVM(x,y):
    model = SVM(max_iter=3, C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data01.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  16.45555090904236
Nilai W :  [30.355604220126242, 22.953045320692365, 47.65070262662166, 20.140019804541858, 19.233768519062636, 32.30847194064101, 20.96560290199995, 11.029839115133237, 8.618334545356607, 16.804466571010693, 6.815502655024715, 7.463610347985349, 9.468334545356607, 54.763872789791805, 5.771113955488956, 5.090820290820291, 10.653518805540868, 2.729532722179781, 35.126023384295394, 24.99056613321317, 6.98727695915196, 9.102904142426205, 8.920993344890407, 2.9128899899120486, 2.7912824308044897, 2.1818831495302082, 1.935972116119175]
Nilai B :  -156.95615871154772


[[6666 2810]
 [1016 4539]]
              precision    recall  f1-score   support

           0       0.87      0.70      0.78      9476
           1       0.62      0.82      0.70      5555

    accuracy  

In [42]:
def PSVM1(x,y):
    model = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM1(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM1(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM1(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM1(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data01.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  17.99166750907898
Nilai W :  [30.87499999999996, 23.47499999999998, 47.9999999999999, 20.94999999999999, 20.324999999999992, 33.049999999999955, 20.34999999999999, 11.575000000000005, 8.650000000000002, 17.25, 6.950000000000002, 7.725000000000001, 9.750000000000004, 55.04999999999988, 5.750000000000001, 5.050000000000001, 10.925000000000006, 3.0999999999999996, 35.049999999999955, 24.39999999999998, 7.375000000000002, 9.075000000000003, 9.325000000000003, 2.9000000000000004, 2.5749999999999997, 2.05, 1.95]
Nilai B :  -158.22243013972033


[[6656 2820]
 [1016 4539]]
              precision    recall  f1-score   support

           0       0.87      0.70      0.78      9476
           1       0.62      0.82      0.70      5555

    accuracy                           0.74     15031
  

# Parallel SVM Corr > 0.2

In [43]:
dataParallel02 = x02_train.copy()
dataParallel02['class'] = y_train.copy()
c1, c2, c3, c4 = splitting_Data(dataParallel02)

x1 = c1.drop(columns=['class']).to_numpy()
y1 = c1['class'].to_numpy()

x2 = c2.drop(columns=['class']).to_numpy()
y2 = c2['class'].to_numpy()

x3 = c3.drop(columns=['class']).to_numpy()
y3 = c3['class'].to_numpy()

x4 = c4.drop(columns=['class']).to_numpy()
y4 = c4['class'].to_numpy()

In [44]:
def PSVM2(x,y):
    model = SVM(max_iter=3, C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM2(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM2(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM2(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM2(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data02.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  10.954708099365234
Nilai W :  [26.974999999999973, 19.749999999999996, 44.624999999999915, 18.175, 18.025, 30.64999999999996, 19.049999999999997, 10.950000000000006, 8.400000000000002, 16.625000000000004, 6.350000000000001, 7.850000000000002, 9.325000000000003, 50.9499999999999]
Nilai B :  -115.02711244178292


[[7017 2459]
 [ 801 4754]]
              precision    recall  f1-score   support

           0       0.90      0.74      0.81      9476
           1       0.66      0.86      0.74      5555

    accuracy                           0.78     15031
   macro avg       0.78      0.80      0.78     15031
weighted avg       0.81      0.78      0.79     15031

Akurasi Model :  78.31 %


In [45]:
def PSVM3(x,y):
    model = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM3(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM3(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM3(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM3(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data02.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  12.487826347351074
Nilai W :  [26.599999999999973, 20.074999999999996, 43.37499999999992, 18.6, 17.6, 29.849999999999966, 18.974999999999994, 10.200000000000005, 8.650000000000004, 16.125000000000004, 6.175000000000001, 6.775000000000001, 9.500000000000004, 49.24999999999991]
Nilai B :  -112.096482035928


[[7017 2459]
 [ 785 4770]]
              precision    recall  f1-score   support

           0       0.90      0.74      0.81      9476
           1       0.66      0.86      0.75      5555

    accuracy                           0.78     15031
   macro avg       0.78      0.80      0.78     15031
weighted avg       0.81      0.78      0.79     15031

Akurasi Model :  78.42 %


# Parallel SVM Corr > 0.3

In [46]:
dataParallel03 = x03_train.copy()
dataParallel03['class'] = y_train.copy()
c1, c2, c3, c4 = splitting_Data(dataParallel03)

x1 = c1.drop(columns=['class']).to_numpy()
y1 = c1['class'].to_numpy()

x2 = c2.drop(columns=['class']).to_numpy()
y2 = c2['class'].to_numpy()

x3 = c3.drop(columns=['class']).to_numpy()
y3 = c3['class'].to_numpy()

x4 = c4.drop(columns=['class']).to_numpy()
y4 = c4['class'].to_numpy()

In [47]:
def PSVM4(x,y):
    model = SVM(max_iter=3, C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM4(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM4(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM4(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM4(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data03.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  8.565078496932983
Nilai W :  [27.724999999999966, 20.074999999999996, 43.82499999999992, 18.825, 18.5, 29.324999999999967, 19.524999999999995]
Nilai B :  -62.807576513639304


[[6759 2717]
 [ 745 4810]]
              precision    recall  f1-score   support

           0       0.90      0.71      0.80      9476
           1       0.64      0.87      0.74      5555

    accuracy                           0.77     15031
   macro avg       0.77      0.79      0.77     15031
weighted avg       0.80      0.77      0.77     15031

Akurasi Model :  76.97 %


In [48]:
def PSVM5(x,y):
    model = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM5(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM5(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM5(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM5(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data03.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  9.885320663452148
Nilai W :  [26.849999999999973, 19.999999999999993, 43.92499999999992, 18.574999999999996, 18.4, 29.999999999999964, 19.624999999999993]
Nilai B :  -62.92607285429133


[[6759 2717]
 [ 745 4810]]
              precision    recall  f1-score   support

           0       0.90      0.71      0.80      9476
           1       0.64      0.87      0.74      5555

    accuracy                           0.77     15031
   macro avg       0.77      0.79      0.77     15031
weighted avg       0.80      0.77      0.77     15031

Akurasi Model :  76.97 %


# Parallel SVM Corr > 0.4

In [49]:
dataParallel04 = x04_train.copy()
dataParallel04['class'] = y_train.copy()
c1, c2, c3, c4 = splitting_Data(dataParallel04)

x1 = c1.drop(columns=['class']).to_numpy()
y1 = c1['class'].to_numpy()

x2 = c2.drop(columns=['class']).to_numpy()
y2 = c2['class'].to_numpy()

x3 = c3.drop(columns=['class']).to_numpy()
y3 = c3['class'].to_numpy()

x4 = c4.drop(columns=['class']).to_numpy()
y4 = c4['class'].to_numpy()

In [50]:
def PSVM6(x,y):
    model = SVM(max_iter=3, C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM6(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM6(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM6(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM6(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data04.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  4.51603889465332
Nilai W :  [37.97499999999994, 27.74999999999997, 40.59999999999993]
Nilai B :  -39.518330006653294


[[4887 4589]
 [ 378 5177]]
              precision    recall  f1-score   support

           0       0.93      0.52      0.66      9476
           1       0.53      0.93      0.68      5555

    accuracy                           0.67     15031
   macro avg       0.73      0.72      0.67     15031
weighted avg       0.78      0.67      0.67     15031

Akurasi Model :  66.95 %


In [51]:
def PSVM7(x,y):
    model = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM7(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM7(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM7(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM7(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data04.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  6.099527597427368
Nilai W :  [37.04999999999994, 26.599999999999973, 40.24999999999993]
Nilai B :  -38.871382235528884


[[4887 4589]
 [ 378 5177]]
              precision    recall  f1-score   support

           0       0.93      0.52      0.66      9476
           1       0.53      0.93      0.68      5555

    accuracy                           0.67     15031
   macro avg       0.73      0.72      0.67     15031
weighted avg       0.78      0.67      0.67     15031

Akurasi Model :  66.95 %


# All Data Positif Correlation

In [52]:
dataParallelPositif = xPositif_train.copy()
dataParallelPositif['class'] = y_train.copy()
c1, c2, c3, c4 = splitting_Data(dataParallelPositif)

x1 = c1.drop(columns=['class']).to_numpy()
y1 = c1['class'].to_numpy()

x2 = c2.drop(columns=['class']).to_numpy()
y2 = c2['class'].to_numpy()

x3 = c3.drop(columns=['class']).to_numpy()
y3 = c3['class'].to_numpy()

x4 = c4.drop(columns=['class']).to_numpy()
y4 = c4['class'].to_numpy()

In [53]:
def PSVM8(x,y):
    model = SVM(max_iter=3, C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM8(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM8(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM8(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM8(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, dataPositif.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  83.64596486091614
Nilai W :  [32.46758639353577, 25.800196551094594, 50.81156133806704, 20.95265549528528, 18.7126674873296, 32.61832554855596, 23.341721156391028, 10.121577631619656, 6.5539113758307135, 8.969069712060946, 16.69789650230652, 7.4882662288934165, 10.031272470095702, 57.93100885639014, 2.3807645324362747, 5.7571082875744874, 3.8251784309543124, 9.076708263944187, 35.13208763706559, 24.727805344100833, 6.538970376294161, 9.749252491043292, 8.706993467407992, 2.4352336421318617, 2.8327001213426986, 1.8782132869398989, 1.8379076589031773, 1.2704155978265415, 2.4580493273575885, 1.018493068428168, 1.5462865345347903, 4.913455215879121, 2.726705827895675, 1.7620032160815327, 2.981363126810024, 1.8849109406108584, 16.95176145523322, 9.523112592070575, 1.7279954884030362, 1.

In [54]:
def PSVM9(x,y):
    model = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM9(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM9(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM9(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM9(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, dataPositif.to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  87.30786204338074
Nilai W :  [37.074999999999946, 28.24999999999997, 57.34999999999988, 24.574999999999978, 22.449999999999985, 37.97499999999994, 25.299999999999976, 12.775000000000002, 7.900000000000002, 9.750000000000004, 20.199999999999992, 8.825000000000003, 10.725000000000003, 64.49999999999986, 3.55, 6.125000000000001, 6.025, 11.475000000000005, 39.57499999999993, 28.224999999999966, 8.425000000000002, 11.025000000000006, 10.250000000000005, 3.4749999999999996, 3.5999999999999996, 2.275, 2.525, 1.55, 2.875, 1.3, 1.675, 5.925000000000001, 3.3000000000000003, 2.0999999999999996, 3.4250000000000003, 2.2, 18.75, 10.900000000000006, 2.125, 2.05, 0.525, 0.575, 8.150000000000002, 6.025, 25.449999999999974, 2.5749999999999997, 6.550000000000001, 1.875, 10.225000000000003, 0.525, 4.5

# All Data Prediction

In [55]:
AllDataParallel = xAll_train.copy()
AllDataParallel['class'] = y_train.copy()
c1, c2, c3, c4 = splitting_Data(AllDataParallel)

x1 = c1.drop(columns=['class']).to_numpy()
y1 = c1['class'].to_numpy()

x2 = c2.drop(columns=['class']).to_numpy()
y2 = c2['class'].to_numpy()

x3 = c3.drop(columns=['class']).to_numpy()
y3 = c3['class'].to_numpy()

x4 = c4.drop(columns=['class']).to_numpy()
y4 = c4['class'].to_numpy()

In [56]:
def PSVM10(x,y):
    model = SVM(max_iter=3, C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM10(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM10(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM10(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM10(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data.drop(columns=['class']).to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  320.82136845588684
Nilai W :  [2.51649500501759, 3.2028960121345817, 3.0833371908487868, 2.4980594185683285, 3.2789872819758514, 5.776384273244233, 33.758199346659424, 1.171130911219291, 1.1958399552423877, 1.6968542203785408, 4.359447306494273, 2.6916083993693642, 26.7229967647915, 47.421353730159936, 0.23646168007363194, 9.747209037116724, 13.200413193259307, 8.8111899385239, 10.672640684212391, 2.7724966895896093, 21.570260688775278, 9.927415496091292, 18.104725464181204, 0.03985435700574171, 7.518541319990647, 29.449657798219906, 0.018682065217391304, 0.028419884035485538, 20.40439179634702, 9.576954668294224, 17.619201393621687, 1.5255653774530216, 14.298640002076606, 9.309456115802284, 16.16869762918212, 6.176888873364131, 0.2997156932455398, 8.669199759032324, 7.760581868745

In [57]:
def PSVM11(x,y):
    model = SVM(max_iter=3, kernel_type='gaussian', C=0.1)
    model.fit(x, y)
    return model.bobotW(), model.bobotB()

# Parallel SVM
start = time.time()

# menampung nilai w dan b dari c1 dan c2
temp=[]
psvm1 = mc.Process(temp.append(PSVM11(x1, y1)))
psvm1.start()
psvm2 = mc.Process(temp.append(PSVM11(x2, y2)))
psvm2.start()
psvm3 = mc.Process(temp.append(PSVM11(x3, y3)))
psvm3.start()
psvm4 = mc.Process(temp.append(PSVM11(x4, y4)))
psvm4.start()
psvm1.join()
psvm2.join()
psvm3.join()
psvm4.join()

# rata-rata nilai w dan b yang akan dipakai
w=[]
for i in range(len(x1[0])):
    w.append(np.mean([temp[0][0][i],temp[1][0][i],temp[2][0][i],temp[3][0][i]]))
b = np.mean([temp[0][1],temp[1][1],temp[2][1],temp[3][1]])

# prediction data
pred = np.sign(np.dot(np.array(w).T, data.drop(columns=['class']).to_numpy().T) + b).astype(int)

# Merubah data -1 menjadi 0
for i in range(len(pred)):
    if(pred[i]==-1):
        pred[i] = 0

stop = time.time()
print('time : ',stop-start)

print('Nilai W : ', w)
print('Nilai B : ', b)
print('\n')
print(confusion_matrix(y, pred))

print(classification_report(y, pred))
print("Akurasi Model : ", round(accuracy_score(y, pred)*100, 2), "%")

Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
Iteration number exceeded the max of 3 iterations
time :  306.912752866745
Nilai W :  [5.575000000000001, 6.975000000000001, 6.775000000000002, 5.550000000000001, 7.050000000000002, 11.325000000000005, 52.82499999999989, 3.4749999999999996, 2.3, 4.1499999999999995, 10.475000000000005, 7.325000000000003, 39.82499999999993, 82.32500000000003, 0.6, 22.024999999999984, 23.249999999999986, 17.150000000000002, 16.700000000000003, 7.175000000000002, 37.099999999999945, 21.099999999999987, 34.549999999999955, 0.125, 18.025, 57.72499999999988, 0.05, 0.125, 41.774999999999935, 22.874999999999982, 35.92499999999995, 4.699999999999999, 28.94999999999997, 22.499999999999986, 35.52499999999995, 15.700000000000006, 1.225, 21.54999999999999, 19.574999999999996, 0.55, 0.025, 11.150000000000004, 14.100000000000005, 29.124999999999968, 38.774999999999935, 1